## Preparación del entorno

In [1]:
# Importar bibliotecas necesarias para el análisis
import pandas as pd

## Carga de datos

### Delitos en la Ciudad
El dataset utilizado en este análisis proviene del [Portal de Datos Abiertos del Gobierno de la Ciudad de Buenos Aires](https://data.buenosaires.gob.ar/dataset/delitos). El dataset abarca el período 2016-2023 Y los datos están organizados en archivos separados por año, donde cada archivo contiene la información de los delitos reportados en ese período específico. Los datos están disponibles bajo la licencia Creative Commons Attribution.

El dataset incluye las siguientes variables:

|Nombre|Tipo|Descripción|
|------|----|-----------|
|id-mapa o id-sum|integer|Identificador único.|
|anio|date|Año en el que se registró el evento.|
|mes|string|Mes en que ocurrió el evento.|
|dia|string|Día de la semana en que ocurrió el evento.|
|fecha|date|Fecha exacta del evento.|
|franja|integer|Franja horaria en la que ocurrió el evento.|
|tipo|string|Clasificación del tipo de delito.|
|subtipo|string|Subtipo del delito, más específico.|
|uso_arma|boolean|Indicador de uso de arma.|
|uso_moto|boolean|Indicador de uso de moto en el evento.|
|barrio|string|Barrio donde ocurrió el evento.|
|comuna|integer|Comuna donde ocurrió el evento.|
|latitud|float|Latitud geográfica donde ocurrió el evento.|
|longitud|float|Longitud geográfica donde ocurrió el evento.|
|cantidad|integer|Número de eventos registrados en esa ubicación y fecha.|

In [64]:
# Lista de archivos que hay que importar (delitos desde 2016 hasta 2023)
# Los archivos están almacenados en la misma carpeta que el notebook Jupyter
archivos_csv = [
    "delitos_2016.csv",
    "delitos_2017.csv",
    "delitos_2018.csv",
    "delitos_2019.csv",
    "delitos_2020.csv",
    "delitos_2021.csv",
    "delitos_2022.csv",
    "delitos_2023.csv"
]

# Crear lista para almacenar todos los data frames
dfs = []

# Importar los archivos de delitos uno por uno
for arch in archivos_csv:
    if arch != "delitos_2023.csv": #El arch de 2023 utiliza ";" como separadores, en lugar de ","
        df = pd.read_csv(arch)
    else:
        df = pd.read_csv(arch, sep=";")
    dfs.append(df)

# Crear el data frame final de delitos
df_delitos = pd.concat(dfs, ignore_index=True) 

# Mostrar primeras filas del data frame de delitos
df_delitos.head()
    

,id-mapa,anio,mes,dia,fecha,franja,tipo,subtipo,uso_arma,uso_moto,barrio,comuna,latitud,longitud,cantidad,id-sum
0,500001.0,2016,ENERO,MARTES,2016-01-26,21.0,Robo,Robo total,NO,NO,VILLA REAL,10.0,-34.617668,-58.530961,1,NaN
1,500004.0,2016,ENERO,MIERCOLES,2016-01-20,16.0,Robo,Robo total,NO,NO,VILLA REAL,10.0,-34.620262,-58.530738,1,NaN
2,500007.0,2016,ENERO,DOMINGO,2016-01-03,13.0,Robo,Robo total,SI,NO,LINIERS,9.0,-34.640094,-58.529826,1,NaN
3,500010.0,2016,ENERO,SABADO,2016-01-09,17.0,Robo,Robo total,NO,NO,LINIERS,9.0,-34.640094,-58.529826,1,NaN
4,500013.0,2016,ENERO,LUNES,2016-01-25,18.0,Robo,Robo total,NO,NO,LINIERS,9.0,-34.640094,-58.529826,1,NaN


### Población de la Ciudad
Para calcular la métrica de "cantidad de delitos por 100.000 personas", es necesario conocer la población de la Ciudad de Buenos Aires a lo largo de los años. Esta información se obtuvo del Instituto de Estadística y Censos de la Ciudad Autónoma de Buenos Aires. Se puede encontrar la información en el siguiente enlace: [Población total estimada por sexo, superficie y densidad poblacional según comuna. Ciudad de Buenos Aires. Años 2006/2023](https://www.estadisticaciudad.gob.ar/eyc/?p=76599)

In [60]:
# Crear diccionario que va a almacenar la población de la Ciudad en cada año
poblacion_ciudad = {
    2016: 3059122,
    2017: 3063728,
    2018: 3068043,
    2019: 3072029,
    2020: 3075646,
    2021: 3078836,
    2022: 3081550,
    2023: 3083770
}

## Limpieza de datos

#### Limpieza de columnas anio, mes, dia y fecha

In [82]:
# Verificar que los años son correctos
anios_invalidos = df_delitos[(df_delitos['anio'] < 2016) | (df_delitos['anio'] > 2023)]
print(f"cantidad de anios invalidos: {len(anios_invalidos)}")

cantidad de anios invalidos: 0


In [83]:
# Verificar que los meses sean válidos
print("meses:", end=" ")
for mes in df_delitos["mes"].unique():
    print(mes, end=" ")
print()

meses: ENERO FEBRERO MARZO ABRIL MAYO JUNIO JULIO AGOSTO SEPTIEMBRE OCTUBRE NOVIEMBRE DICIEMBRE 


In [84]:
# Verificar que los días de la semana sean válidos
print("dias de la semana:", end=" ")
for dia in df_delitos["dia"].unique():
    print(dia, end=" ")
print()

dias de la semana: MARTES MIERCOLES DOMINGO SABADO LUNES VIERNES JUEVES 


In [85]:
# Se detectó un error en los datos: Los días "SÁBADO" y "MIÉRCOLES" aparecen con y sin tilde
# Para solucionar esto, se normalizan los valores eliminado los tildes para mantener la consistencia
df_delitos.loc[df_delitos["dia"] == "SÁBADO", "dia"] = "SABADO"
df_delitos.loc[df_delitos["dia"] == "MIÉRCOLES", "dia"] = "MIERCOLES"

# Verificar los cambios
print("dias de la semana:", end=" ")
for dia in df_delitos["dia"].unique():
    print(dia, end=" ")
print()

dias de la semana: MARTES MIERCOLES DOMINGO SABADO LUNES VIERNES JUEVES 


In [102]:
# Convertir columnas 'uso_arma' y 'uso_moto' de SI/NO a valores booleanos
# para facilitar el análisis
bool_conversion = {"SI": True, "NO": False}
df_delitos["uso_arma"] = df_delitos["uso_arma"].map(bool_conversion)
df_delitos["uso_moto"] = df_delitos["uso_moto"].map(bool_conversion)

# Verifico los cambios
print(df_delitos["uso_arma"].unique())
print(df_delitos["uso_moto"].unique())


[False  True]
[False  True]


In [103]:
# Asignar el tipo de dato correcto a las columnas
columnas_dtype= {
    'id': 'Int64',
    'mes': 'string',
    'dia': 'string',
    'franja': 'Int64',
    'tipo': 'string',
    'subtipo': 'string',
    'uso_arma': 'boolean',
    'uso_moto': 'boolean',
    'barrio': 'string',
    'comuna': 'Int64',
    'latitud': 'float64',
    'longitud': 'float64',
    'cantidad': 'Int64'
}

df_delitos = df_delitos.astype(columnas_dtype)

# Verificar tipos de datos
df_delitos.dtypes

ValueError: invalid literal for int() with base 10: 'CC-08': Error while type casting for column 'comuna'

In [105]:
# Algunos archivos utilizan "id-mapa" como id y otros utilizan "id-sum".
# Crear columna que almacene los ids de los delitos reportados
if 'id' not in df_delitos.columns:
    df_delitos['id'] = df_delitos['id-mapa'].fillna(df_delitos['id-sum'])

# Verificar que no halla ids faltantes
print(f"Ids faltantes: {df_delitos['id'].isna().sum()}")

# Eliminar columnas "id-mapa" y "id-sum"
if 'id-mapa' in df_delitos.columns:
    df_delitos.drop('id-mapa', axis=1, inplace=True)
if 'id-sum' in df_delitos.columns:
    df_delitos.drop('id-sum', axis=1, inplace=True)


Ids faltantes: 0


In [68]:
# Hallar si hay registros repetidos
print(f"delitos duplicados: {df_delitos.duplicated().sum()}")

delitos duplicados: 0


In [81]:
# Ver el tipo de dato de las columnas
df_delitos.dtypes

id-mapa     float64
anio          int64
mes          object
dia          object
fecha        object
franja      float64
tipo         object
subtipo      object
uso_arma     object
uso_moto     object
barrio       object
comuna       object
latitud      object
longitud     object
cantidad      int64
id-sum      float64
id          float64
dtype: object

### Verificación de las fechas

In [107]:
df_delitos.head()

[10.0 9.0 11.0 12.0 15.0 7.0 8.0 13.0 6.0 14.0 5.0 4.0 nan 2.0 3.0 1.0
 '13' '8' '12' '7' '3' '2' '10' '4' '15' '9' '14' '1' '11' '5' '6' 'CC-08'
 'CC-09' 'CC-01 NORTE' 'CC-04' 'CC-07' 'CC-15' 'CC-02' 'CC-12' 'CC-10'
 'CC-06' 'CC-13' 'CC-05' 'CC-01 SUR' 'CC-03' 'CC-14' 'Sin geo' 'CC-11']
